---
---

# **`CGM - Assignment` : Predicting Height of the Children 👧👦📏** 

- Malnutrition is a global health crisis and is a leading cause of death among children under 5 years. 

- Detecting malnutrition requires `anthropometric measurements` of weight, height, and middle-upper arm circumference. 
    - Anthropometric measurements are a non invasive, inexpensive, and suitable method for `evaluating the nutritional status` in the samples.
    - However, anthropometric techniques are `prone to errors` that could arise, for example, from the inadequate training of personnel.


- Therefore, using a CNN-based deep learning model can certainly help in faster, easier and more precised collection of such measurements.


---
---

## **`Examples of various measurements` are shown below:**
![img_1](E:\STUDY\PROJECTS\Assignment_CGM\Doc_Images\Anthropometric_Measurements.png)


---
---


## **`NOTEBOOK Description` 📗**

- This notebook contains the below tasks:
    - `Loading the Model`
        - Importing the model architecture and loading the best weights
    - `Generating the DL features`
        - From the Resnet-50 model, generating the 2048 dim features for every given image
    - `Generating predictions from ML model using above DL features`
        - Loading the RF Regressor model
        - Generating predictions for every provided image

## **`NOTEBOOK Contents` ✍️**

[1. Libraries Import](#Import_Libraries)

[2. Dataset Import](#Dataset_Import)

##### ⭐ **`Generating DL Features`**

[3. 🧮 For every given image generating the DL Features.](#Generating_DL_Features)

##### ⭐ **`Loading Best Model`**

[4. 🧮 Loading the model as per the 3rd strategy mentioned in 2nd notebook.](#Loading_best_model)

##### ⭐ **`Generating Predictions`**

[5. 🧮 Generating the predictions for every given image.](#Generating_Predictions)


[⭐ SUMMARY](#SUMMARY)

## Import_Libraries

In [1]:
import os
import sys
import math
import scipy as scipy
import numpy as np
from numpy import arange
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import ast
import pathlib

import plotly.express as px
import plotly.graph_objects as go
import hashlib
from IPython.display import display
from PIL import Image
from tqdm import tqdm
tqdm.pandas()

%matplotlib inline

In [2]:
# Setting the display option
pd.set_option('display.max_columns', 50)

# Defining the variables for better plotting 
label_font_dict = {'family':'sans-serif','size':13.5,'color':'brown','style':'italic'}
title_font_dict = {'family':'sans-serif','size':17.5,'color':'Blue','style':'italic'}

## **Dataset_Import**

In [3]:
!pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [4]:
## TRAIN set files
!gdown 1kuGJodL48MaAb57j2VSbSX_qXellGrBC
!gdown 1NAwb73kvK_uut71vNIte34tiaDKfSmTZ

Downloading...
From: https://drive.google.com/uc?id=1kuGJodL48MaAb57j2VSbSX_qXellGrBC
To: /content/Dataset.zip
100% 203k/203k [00:00<00:00, 105MB/s]
Downloading...
From: https://drive.google.com/uc?id=1NAwb73kvK_uut71vNIte34tiaDKfSmTZ
To: /content/HP_data_engg.csv
100% 3.55k/3.55k [00:00<00:00, 5.49MB/s]


In [5]:
!unzip Dataset.zip

Archive:  Dataset.zip
  inflating: Dataset/1d5fee00-ada4-11eb-a80b-3f50af21830f.jpg  
  inflating: Dataset/1eddf4c0-ad9f-11eb-a80b-ef4adba049aa.jpg  
  inflating: Dataset/38e29bc0-8fc0-11eb-b5f2-0742da91b282.jpg  
  inflating: Dataset/6cd5d480-8b8b-11eb-b152-f7b115384fa9.jpg  
  inflating: Dataset/76ed6fc0-8e1d-11eb-ad70-7be1a1219834.jpg  
  inflating: Dataset/8d23fa60-9201-11eb-b3e6-cbfb48c6a300.jpg  
  inflating: Dataset/9dc3eba0-8d8d-11eb-8d3f-4776be867c61.jpg  
  inflating: Dataset/9dd020a0-8d8d-11eb-8d3f-bfd9f6bd04d4.jpg  
  inflating: Dataset/d3757800-8dff-11eb-a328-472d4f8db277.jpg  


In [6]:
# Storing the location of the 'height_and_pose' file
hp_df = pd.read_csv(os.path.join(os.getcwd(), "HP_data_engg.csv"), index_col="Unnamed: 0")
hp_df['Img_name'] = hp_df['Img_name'].apply(lambda name: os.path.join(os.path.join(os.getcwd(), "Dataset"), name+".jpg"))
hp_df

,Img_name,nose_top_dist,nose_bottom_dist,left_eye_top_dist,left_eye_bottom_dist,right_eye_top_dist,right_eye_bottom_dist,left_ear_top_dist,left_ear_bottom_dist,right_ear_top_dist,right_ear_bottom_dist,left_shoulder_top_dist,left_shoulder_bottom_dist,right_shoulder_top_dist,right_shoulder_bottom_dist,left_elbow_top_dist,left_elbow_bottom_dist,right_elbow_top_dist,right_elbow_bottom_dist,left_wrist_top_dist,left_wrist_bottom_dist,right_wrist_top_dist,right_wrist_bottom_dist,left_hip_top_dist,left_hip_bottom_dist,right_hip_top_dist,right_hip_bottom_dist,left_knee_top_dist,left_knee_bottom_dist,right_knee_top_dist,right_knee_bottom_dist,left_ankle_top_dist,left_ankle_bottom_dist,right_ankle_top_dist,right_ankle_bottom_dist,Height
0,/content/Dataset/1d5fee00-ada4-11eb-a80b-3f50a...,139.691,453.810,151.440,460.771,113.032,476.286,180.259,445.126,101.246,478.053,224.946,388.402,163.677,412.205,291.861,332.618,218.935,368.847,319.802,276.817,235.590,342.005,352.147,229.775,330.627,247.948,457.988,118.041,400.249,210.554,426.089,152.056,458.407,194.857,76.7
1,/content/Dataset/1eddf4c0-ad9f-11eb-a80b-ef4ad...,219.240,376.907,107.062,464.210,208.241,395.238,139.040,432.574,202.198,386.158,170.682,399.706,237.697,349.583,130.723,440.394,287.228,340.067,95.316,481.054,291.823,375.793,306.436,272.108,332.235,238.045,374.111,226.972,393.302,177.855,447.702,165.827,464.532,105.850,66.4
2,/content/Dataset/38e29bc0-8fc0-11eb-b5f2-0742d...,114.684,464.978,121.393,460.750,209.028,415.961,139.339,438.516,210.612,412.764,180.103,396.358,244.709,353.241,117.177,462.027,254.459,392.036,107.047,483.412,254.249,442.271,334.531,250.052,365.415,211.029,409.651,202.716,429.890,146.674,489.473,148.322,489.456,103.502,63.2
3,/content/Dataset/6cd5d480-8b8b-11eb-b152-f7b11...,150.096,505.597,155.437,516.805,127.744,524.638,185.402,504.764,125.420,520.907,242.182,436.003,178.191,464.880,308.353,359.789,245.235,406.229,362.151,296.329,311.701,352.524,369.434,274.015,348.848,301.257,464.828,182.184,445.466,225.565,543.030,128.115,531.762,177.651,87.1
4,/content/Dataset/76ed6fc0-8e1d-11eb-ad70-7be1a...,91.655,500.699,88.932,502.501,89.834,501.079,122.090,470.116,167.362,470.016,160.697,430.627,194.623,439.693,129.537,468.497,212.710,471.281,84.680,507.193,183.968,515.070,308.260,285.770,327.234,268.397,381.836,239.404,422.678,168.536,479.981,165.535,507.097,90.993,59.1
5,/content/Dataset/8d23fa60-9201-11eb-b3e6-cbfb4...,190.423,472.569,179.159,487.789,172.724,488.188,131.322,488.105,136.639,483.716,169.193,435.949,209.694,408.771,239.258,375.096,310.914,304.460,271.681,333.434,289.150,337.040,329.403,282.248,353.680,251.499,447.742,172.448,467.127,140.664,547.518,117.861,567.176,64.311,86.4
6,/content/Dataset/9dc3eba0-8d8d-11eb-8d3f-4776b...,142.212,458.133,136.621,473.452,116.796,480.265,155.741,472.965,103.867,489.563,221.702,394.308,180.226,415.960,300.493,308.431,263.732,342.875,364.402,238.180,337.507,279.494,329.515,265.448,312.568,285.209,422.124,171.750,414.752,195.807,488.571,113.838,492.155,138.798,100.6
7,/content/Dataset/9dd020a0-8d8d-11eb-8d3f-bfd9f...,118.411,437.884,110.964,454.636,96.762,457.973,135.269,443.555,101.644,451.652,199.977,375.700,158.455,396.762,280.915,285.278,247.099,316.309,344.566,216.070,329.922,244.375,325.652,228.244,312.485,246.740,409.378,146.260,405.849,163.278,479.062,91.341,475.298,110.771,106.1
8,/content/Dataset/d3757800-8dff-11eb-a328-472d4...,179.026,501.837,174.395,509.670,176.691,516.647,111.132,530.799,170.003,517.505,173.186,468.816,238.792,433.049,277.820,377.785,333.128,325.585,348.933,315.665,400.402,253.969,354.905,290.880,378.720,261.624,464.386,197.124,471.840,167.764,571.270,159.135,583.359,62.913,99.9


In [7]:
hp_df.shape

(9, 36)

## **`Data_Preparation`**

In [8]:
import random as rn
import datetime
import time
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, Activation, Flatten, Conv1D, Conv2D, Concatenate, Lambda
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import MaxPool1D, MaxPool2D, GlobalAveragePooling1D, GlobalAveragePooling2D, AveragePooling1D, AveragePooling2D, SpatialDropout2D
from keras.regularizers import l1, l2
from keras.preprocessing import image
from keras.callbacks import Callback
from keras.callbacks import TensorBoard
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, roc_auc_score, confusion_matrix
from tensorboard import notebook
from tensorflow.keras.models import model_from_json
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Loading the required libraries
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
import json

In [9]:
# Downloading the Model Arch & Best Weights
!gdown 1YJWChf3MmiiKs86fpXww6_1jnoY68Jlb
!gdown 1sTuiWCHjKQJIjAHigapEFCZ1bKXqzJZw

Downloading...
From: https://drive.google.com/uc?id=1YJWChf3MmiiKs86fpXww6_1jnoY68Jlb
To: /content/resnet_50b.json
100% 86.6k/86.6k [00:00<00:00, 43.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=1sTuiWCHjKQJIjAHigapEFCZ1bKXqzJZw
To: /content/weights-30-193.21.hdf5
100% 96.6M/96.6M [00:01<00:00, 77.4MB/s]


In [ ]:
# loading Models' JSON and creating the Model
with open('resnet_50b.json', 'r') as model_json_file:
    loaded_model_json = model_json_file.read()
    
# Loading the model from json
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("weights-30-193.21.hdf5")

In [ ]:
# Storing the DL Model with top layers excluded
new_model = tf.keras.models.Model(loaded_model.input, 
                                  loaded_model.get_layer('Flatten_for_hidden_layers').output)

In [ ]:
hp_df

,Img_name,nose_top_dist,nose_bottom_dist,left_eye_top_dist,left_eye_bottom_dist,right_eye_top_dist,right_eye_bottom_dist,left_ear_top_dist,left_ear_bottom_dist,right_ear_top_dist,right_ear_bottom_dist,left_shoulder_top_dist,left_shoulder_bottom_dist,right_shoulder_top_dist,right_shoulder_bottom_dist,left_elbow_top_dist,left_elbow_bottom_dist,right_elbow_top_dist,right_elbow_bottom_dist,left_wrist_top_dist,left_wrist_bottom_dist,right_wrist_top_dist,right_wrist_bottom_dist,left_hip_top_dist,left_hip_bottom_dist,right_hip_top_dist,right_hip_bottom_dist,left_knee_top_dist,left_knee_bottom_dist,right_knee_top_dist,right_knee_bottom_dist,left_ankle_top_dist,left_ankle_bottom_dist,right_ankle_top_dist,right_ankle_bottom_dist,Height
0,/content/Dataset/1d5fee00-ada4-11eb-a80b-3f50a...,139.691,453.810,151.440,460.771,113.032,476.286,180.259,445.126,101.246,478.053,224.946,388.402,163.677,412.205,291.861,332.618,218.935,368.847,319.802,276.817,235.590,342.005,352.147,229.775,330.627,247.948,457.988,118.041,400.249,210.554,426.089,152.056,458.407,194.857,76.7
1,/content/Dataset/1eddf4c0-ad9f-11eb-a80b-ef4ad...,219.240,376.907,107.062,464.210,208.241,395.238,139.040,432.574,202.198,386.158,170.682,399.706,237.697,349.583,130.723,440.394,287.228,340.067,95.316,481.054,291.823,375.793,306.436,272.108,332.235,238.045,374.111,226.972,393.302,177.855,447.702,165.827,464.532,105.850,66.4
2,/content/Dataset/38e29bc0-8fc0-11eb-b5f2-0742d...,114.684,464.978,121.393,460.750,209.028,415.961,139.339,438.516,210.612,412.764,180.103,396.358,244.709,353.241,117.177,462.027,254.459,392.036,107.047,483.412,254.249,442.271,334.531,250.052,365.415,211.029,409.651,202.716,429.890,146.674,489.473,148.322,489.456,103.502,63.2
3,/content/Dataset/6cd5d480-8b8b-11eb-b152-f7b11...,150.096,505.597,155.437,516.805,127.744,524.638,185.402,504.764,125.420,520.907,242.182,436.003,178.191,464.880,308.353,359.789,245.235,406.229,362.151,296.329,311.701,352.524,369.434,274.015,348.848,301.257,464.828,182.184,445.466,225.565,543.030,128.115,531.762,177.651,87.1
4,/content/Dataset/76ed6fc0-8e1d-11eb-ad70-7be1a...,91.655,500.699,88.932,502.501,89.834,501.079,122.090,470.116,167.362,470.016,160.697,430.627,194.623,439.693,129.537,468.497,212.710,471.281,84.680,507.193,183.968,515.070,308.260,285.770,327.234,268.397,381.836,239.404,422.678,168.536,479.981,165.535,507.097,90.993,59.1
5,/content/Dataset/8d23fa60-9201-11eb-b3e6-cbfb4...,190.423,472.569,179.159,487.789,172.724,488.188,131.322,488.105,136.639,483.716,169.193,435.949,209.694,408.771,239.258,375.096,310.914,304.460,271.681,333.434,289.150,337.040,329.403,282.248,353.680,251.499,447.742,172.448,467.127,140.664,547.518,117.861,567.176,64.311,86.4
6,/content/Dataset/9dc3eba0-8d8d-11eb-8d3f-4776b...,142.212,458.133,136.621,473.452,116.796,480.265,155.741,472.965,103.867,489.563,221.702,394.308,180.226,415.960,300.493,308.431,263.732,342.875,364.402,238.180,337.507,279.494,329.515,265.448,312.568,285.209,422.124,171.750,414.752,195.807,488.571,113.838,492.155,138.798,100.6
7,/content/Dataset/9dd020a0-8d8d-11eb-8d3f-bfd9f...,118.411,437.884,110.964,454.636,96.762,457.973,135.269,443.555,101.644,451.652,199.977,375.700,158.455,396.762,280.915,285.278,247.099,316.309,344.566,216.070,329.922,244.375,325.652,228.244,312.485,246.740,409.378,146.260,405.849,163.278,479.062,91.341,475.298,110.771,106.1
8,/content/Dataset/d3757800-8dff-11eb-a328-472d4...,179.026,501.837,174.395,509.670,176.691,516.647,111.132,530.799,170.003,517.505,173.186,468.816,238.792,433.049,277.820,377.785,333.128,325.585,348.933,315.665,400.402,253.969,354.905,290.880,378.720,261.624,464.386,197.124,471.840,167.764,571.270,159.135,583.359,62.913,99.9


## **`Generating_DL_Features`**

In [ ]:
def decoding_image(img_name, tgt_label=None, image_size=(448, 448)):
    """
    Description : This function is created for decoding, normalizing and resizing the input image.
    
    Inputs : It accepts below 3 parameters:
        - img_name : Image name
        - label : Target label of that image
        - image_size : New size of the image
    
    Returns : It returns the tensor object of an image and its tgt label.
    """
    # Reading the image as a tensor of string
    img_bytes_str = tf.io.read_file(img_name)
    
    # Detects whether an image is a BMP, GIF, JPEG, or PNG, and, 
    # performs the appropriate operation to convert the input bytes string into a Tensor of type dtype.
    image_tensor = tf.image.decode_jpeg(img_bytes_str, channels=3)
    
    # Casting the img tensor to a float type and normalizing it
    image_tensor = tf.cast(image_tensor, tf.float32) / 255.0
    
    # Re-sizing the image
    image_tensor = tf.image.resize(image_tensor, image_size)
    
    # If respective target label is provided then returns it else return the image tensor
    if tgt_label is None:
        return image_tensor
    else:
        return image_tensor, tgt_label

In [ ]:
# Generating the features for one of the images
features_x_train = new_model.predict(np.expand_dims(decoding_image(hp_df['Img_name'].iloc[0]).numpy(), axis=0))

1/1 [==============================] - 2s 2s/step


In [ ]:
# Checking the shape of the DL features
features_x_train.shape

(1, 2048)

In [ ]:
# Displaying the features
features_x_train

array([[0.       , 0.       , 0.       , ..., 2.1255698, 0.       ,
        0.       ]], dtype=float32)

In [ ]:
# Generating the features for all the images
imgs_dl_feats = hp_df['Img_name'].apply(lambda img: new_model.predict(np.expand_dims(decoding_image(img).numpy(), axis=0))[0])

1/1 [==============================] - 1s 712ms/step


In [ ]:
# Defining some column name & checking the total
cols = ["f"+ str(i) for i in range(1,2049)]
len(cols)

2048

In [ ]:
# Storing every dim value as a column in the dataframe
imgs_dl_feats_df = pd.DataFrame(imgs_dl_feats)['Img_name'].apply(pd.Series)
imgs_dl_feats_df.columns = cols
all_imgs_feats_df = pd.concat([pd.DataFrame(hp_df['Img_name']), imgs_dl_feats_df, pd.DataFrame(hp_df['Height'])], axis=1)
all_imgs_feats_df

,Img_name,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,...,f2025,f2026,f2027,f2028,f2029,f2030,f2031,f2032,f2033,f2034,f2035,f2036,f2037,f2038,f2039,f2040,f2041,f2042,f2043,f2044,f2045,f2046,f2047,f2048,Height
0,/content/Dataset/1d5fee00-ada4-11eb-a80b-3f50a...,0.0,0.0,0.000000,0.0,4.919989,0.0,0.0,1.232318,0.041064,0.0,0.0,0.0,1.211448,0.596881,5.314107,0.068448,0.270772,0.0,0.000000,0.671996,0.0,0.0,1.887564,0.234780,...,0.425136,0.046488,0.000000,0.0,0.0,0.0,0.000000,4.125062,0.069655,0.0,0.016122,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.443612,0.0,0.0,2.125570,0.0,0.0,76.7
1,/content/Dataset/1eddf4c0-ad9f-11eb-a80b-ef4ad...,0.0,0.0,0.000000,0.0,4.897092,0.0,0.0,1.242595,0.040788,0.0,0.0,0.0,1.194960,0.585660,5.247516,0.052826,0.267805,0.0,0.000000,0.678129,0.0,0.0,1.891366,0.228880,...,0.457143,0.042686,0.000000,0.0,0.0,0.0,0.000000,4.096988,0.073125,0.0,0.015152,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.461200,0.0,0.0,2.153026,0.0,0.0,66.4
2,/content/Dataset/38e29bc0-8fc0-11eb-b5f2-0742d...,0.0,0.0,0.001588,0.0,4.898275,0.0,0.0,1.250681,0.036835,0.0,0.0,0.0,1.215819,0.592546,5.262149,0.067028,0.265735,0.0,0.000000,0.668743,0.0,0.0,1.917466,0.237922,...,0.412653,0.049392,0.000000,0.0,0.0,0.0,0.000000,4.134161,0.076137,0.0,0.017131,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.444417,0.0,0.0,2.189022,0.0,0.0,63.2
3,/content/Dataset/6cd5d480-8b8b-11eb-b152-f7b11...,0.0,0.0,0.000173,0.0,4.908416,0.0,0.0,1.267020,0.051824,0.0,0.0,0.0,1.207435,0.605955,5.256791,0.065130,0.266906,0.0,0.000000,0.666541,0.0,0.0,1.939418,0.235997,...,0.407205,0.048160,0.000000,0.0,0.0,0.0,0.000000,4.132693,0.070295,0.0,0.022843,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.420259,0.0,0.0,2.175637,0.0,0.0,87.1
4,/content/Dataset/76ed6fc0-8e1d-11eb-ad70-7be1a...,0.0,0.0,0.001794,0.0,4.928632,0.0,0.0,1.250752,0.036022,0.0,0.0,0.0,1.212313,0.586105,5.260494,0.060708,0.266331,0.0,0.000000,0.676268,0.0,0.0,1.914697,0.241128,...,0.431022,0.046676,0.000000,0.0,0.0,0.0,0.000000,4.134078,0.080924,0.0,0.018746,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.461155,0.0,0.0,2.197828,0.0,0.0,59.1
5,/content/Dataset/8d23fa60-9201-11eb-b3e6-cbfb4...,0.0,0.0,0.000000,0.0,4.913174,0.0,0.0,1.265471,0.058285,0.0,0.0,0.0,1.194981,0.592711,5.341297,0.096545,0.268521,0.0,0.000000,0.671043,0.0,0.0,1.756436,0.278063,...,0.364584,0.016856,0.000000,0.0,0.0,0.0,0.003713,4.169618,0.154638,0.0,0.028241,0.0,0.000196,0.0,0.0,0.0,0.000671,0.0,0.501747,0.0,0.0,2.107327,0.0,0.0,86.4
6,/content/Dataset/9dc3eba0-8d8d-11eb-8d3f-4776b...,0.0,0.0,0.000000,0.0,4.798597,0.0,0.0,1.260125,0.049902,0.0,0.0,0.0,1.235800,0.588496,5.168211,0.090648,0.275804,0.0,0.004053,0.647746,0.0,0.0,1.716465,0.281806,...,0.423575,0.012758,0.000069,0.0,0.0,0.0,0.030556,4.030042,0.149440,0.0,0.026023,0.0,0.000627,0.0,0.0,0.0,0.005098,0.0,0.550922,0.0,0.0,2.063637,0.0,0.0,100.6
7,/content/Dataset/9dd020a0-8d8d-11eb-8d3f-bfd9f...,0.0,0.0,0.000000,0.0,4.870487,0.0,0.0,1.277971,0.044576,0.0,0.0,0.0,1.213791,0.658078,5.334865,0.096003,0.293912,0.0,0.000085,0.675494,0.0,0.0,1.734279,0.266230,...,0.395978,0.020612,0.000000,0.0,0.0,0.0,0.029332,4.095998,0.108282,0.0,0.017380,0.0,0.000659,0.0,0.0,0.0,0.004383,0.0,0.498547,0.0,0.0,2.136361,0.0,0.0,106.1
8,/content/Dataset/d3757800-8dff-11eb-a328-472d4...,0.0,0.0,0.000000,0.0,5.007003,0.0,0.0,1.264733,0.048589,0.0,0.0,0.0,1.196861,0.618300,5.311005,0.140203,0.296374,0.0,0.002209,0.700000,0.0,0.0,1.827778,0.292010,...,0.392334,0.009061,0.000000,0.0,0.0,0.0,0.014609,4.082306,0.119844,0.0,0.024264,0.0,0.000000,0.0,0.0,0.0,0.002515,0.0,0.588309,0.0,0.0,2.131566,0.0,0.0,99.9


In [ ]:
# Storing only image name
all_imgs_feats_df['Img_name'] = all_imgs_feats_df['Img_name'].apply(lambda val: val.split("/")[-1])

In [ ]:
# Displaying the content
all_imgs_feats_df

,Img_name,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,...,f2025,f2026,f2027,f2028,f2029,f2030,f2031,f2032,f2033,f2034,f2035,f2036,f2037,f2038,f2039,f2040,f2041,f2042,f2043,f2044,f2045,f2046,f2047,f2048,Height
0,1d5fee00-ada4-11eb-a80b-3f50af21830f.jpg,0.0,0.0,0.000000,0.0,4.919989,0.0,0.0,1.232318,0.041064,0.0,0.0,0.0,1.211448,0.596881,5.314107,0.068448,0.270772,0.0,0.000000,0.671996,0.0,0.0,1.887564,0.234780,...,0.425136,0.046488,0.000000,0.0,0.0,0.0,0.000000,4.125062,0.069655,0.0,0.016122,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.443612,0.0,0.0,2.125570,0.0,0.0,76.7
1,1eddf4c0-ad9f-11eb-a80b-ef4adba049aa.jpg,0.0,0.0,0.000000,0.0,4.897092,0.0,0.0,1.242595,0.040788,0.0,0.0,0.0,1.194960,0.585660,5.247516,0.052826,0.267805,0.0,0.000000,0.678129,0.0,0.0,1.891366,0.228880,...,0.457143,0.042686,0.000000,0.0,0.0,0.0,0.000000,4.096988,0.073125,0.0,0.015152,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.461200,0.0,0.0,2.153026,0.0,0.0,66.4
2,38e29bc0-8fc0-11eb-b5f2-0742da91b282.jpg,0.0,0.0,0.001588,0.0,4.898275,0.0,0.0,1.250681,0.036835,0.0,0.0,0.0,1.215819,0.592546,5.262149,0.067028,0.265735,0.0,0.000000,0.668743,0.0,0.0,1.917466,0.237922,...,0.412653,0.049392,0.000000,0.0,0.0,0.0,0.000000,4.134161,0.076137,0.0,0.017131,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.444417,0.0,0.0,2.189022,0.0,0.0,63.2
3,6cd5d480-8b8b-11eb-b152-f7b115384fa9.jpg,0.0,0.0,0.000173,0.0,4.908416,0.0,0.0,1.267020,0.051824,0.0,0.0,0.0,1.207435,0.605955,5.256791,0.065130,0.266906,0.0,0.000000,0.666541,0.0,0.0,1.939418,0.235997,...,0.407205,0.048160,0.000000,0.0,0.0,0.0,0.000000,4.132693,0.070295,0.0,0.022843,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.420259,0.0,0.0,2.175637,0.0,0.0,87.1
4,76ed6fc0-8e1d-11eb-ad70-7be1a1219834.jpg,0.0,0.0,0.001794,0.0,4.928632,0.0,0.0,1.250752,0.036022,0.0,0.0,0.0,1.212313,0.586105,5.260494,0.060708,0.266331,0.0,0.000000,0.676268,0.0,0.0,1.914697,0.241128,...,0.431022,0.046676,0.000000,0.0,0.0,0.0,0.000000,4.134078,0.080924,0.0,0.018746,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.461155,0.0,0.0,2.197828,0.0,0.0,59.1
5,8d23fa60-9201-11eb-b3e6-cbfb48c6a300.jpg,0.0,0.0,0.000000,0.0,4.913174,0.0,0.0,1.265471,0.058285,0.0,0.0,0.0,1.194981,0.592711,5.341297,0.096545,0.268521,0.0,0.000000,0.671043,0.0,0.0,1.756436,0.278063,...,0.364584,0.016856,0.000000,0.0,0.0,0.0,0.003713,4.169618,0.154638,0.0,0.028241,0.0,0.000196,0.0,0.0,0.0,0.000671,0.0,0.501747,0.0,0.0,2.107327,0.0,0.0,86.4
6,9dc3eba0-8d8d-11eb-8d3f-4776be867c61.jpg,0.0,0.0,0.000000,0.0,4.798597,0.0,0.0,1.260125,0.049902,0.0,0.0,0.0,1.235800,0.588496,5.168211,0.090648,0.275804,0.0,0.004053,0.647746,0.0,0.0,1.716465,0.281806,...,0.423575,0.012758,0.000069,0.0,0.0,0.0,0.030556,4.030042,0.149440,0.0,0.026023,0.0,0.000627,0.0,0.0,0.0,0.005098,0.0,0.550922,0.0,0.0,2.063637,0.0,0.0,100.6
7,9dd020a0-8d8d-11eb-8d3f-bfd9f6bd04d4.jpg,0.0,0.0,0.000000,0.0,4.870487,0.0,0.0,1.277971,0.044576,0.0,0.0,0.0,1.213791,0.658078,5.334865,0.096003,0.293912,0.0,0.000085,0.675494,0.0,0.0,1.734279,0.266230,...,0.395978,0.020612,0.000000,0.0,0.0,0.0,0.029332,4.095998,0.108282,0.0,0.017380,0.0,0.000659,0.0,0.0,0.0,0.004383,0.0,0.498547,0.0,0.0,2.136361,0.0,0.0,106.1
8,d3757800-8dff-11eb-a328-472d4f8db277.jpg,0.0,0.0,0.000000,0.0,5.007003,0.0,0.0,1.264733,0.048589,0.0,0.0,0.0,1.196861,0.618300,5.311005,0.140203,0.296374,0.0,0.002209,0.700000,0.0,0.0,1.827778,0.292010,...,0.392334,0.009061,0.000000,0.0,0.0,0.0,0.014609,4.082306,0.119844,0.0,0.024264,0.0,0.000000,0.0,0.0,0.0,0.002515,0.0,0.588309,0.0,0.0,2.131566,0.0,0.0,99.9


In [ ]:
# Checking the type of values
type(all_imgs_feats_df.values)

numpy.ndarray

In [ ]:
# Storing the DL features for all the images
np.savez("all_imgs_df_feats.npz",all_imgs_feats_df.values)

In [33]:
# mount the drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# copy it there
!cp all_imgs_df_feats.npz /content/drive/MyDrive/CGM_Assignment

In [10]:
# Downloading all the DL features and BEST model
!gdown 1RPTOAkX5imYTHP8144C1B6EiY6TXh0s0
!gdown 1EoTtDI56PMxPHNB_VQWH9PmxQj9qybg5

Downloading...
From: https://drive.google.com/uc?id=1RPTOAkX5imYTHP8144C1B6EiY6TXh0s0
To: /content/all_imgs_df_feats.npz
100% 167k/167k [00:00<00:00, 61.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1EoTtDI56PMxPHNB_VQWH9PmxQj9qybg5
To: /content/rf_model_jlib
100% 12.8k/12.8k [00:00<00:00, 10.3MB/s]


## **Loading_best_model**

In [11]:
# loading the best model file
loaded_rf = joblib.load('rf_model_jlib')

In [12]:
# Defining some columns
cols = ["f"+ str(i) for i in range(1,2049)]
len(cols)

2048

In [13]:
# Adding Img name & height col name
cols.insert(0,'Img_name')
cols.insert(len(cols),'Height')

In [14]:
# Checking the total cols count
len(cols)

2050

In [15]:
# Loading the Data
all_imgs_feats_file = np.load("all_imgs_df_feats.npz", allow_pickle=True)
all_imgs_feats = all_imgs_feats_file.f.arr_0

In [16]:
data_df = pd.DataFrame(all_imgs_feats, columns = cols)
data_df

,Img_name,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,...,f2025,f2026,f2027,f2028,f2029,f2030,f2031,f2032,f2033,f2034,f2035,f2036,f2037,f2038,f2039,f2040,f2041,f2042,f2043,f2044,f2045,f2046,f2047,f2048,Height
0,1d5fee00-ada4-11eb-a80b-3f50af21830f.jpg,0.0,0.0,0.0,0.0,4.919989,0.0,0.0,1.232318,0.041064,0.0,0.0,0.0,1.211448,0.596881,5.314107,0.068448,0.270772,0.0,0.0,0.671996,0.0,0.0,1.887564,0.23478,...,0.425136,0.046488,0.0,0.0,0.0,0.0,0.0,4.125062,0.069655,0.0,0.016122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.443612,0.0,0.0,2.12557,0.0,0.0,76.7
1,1eddf4c0-ad9f-11eb-a80b-ef4adba049aa.jpg,0.0,0.0,0.0,0.0,4.897092,0.0,0.0,1.242595,0.040788,0.0,0.0,0.0,1.19496,0.58566,5.247516,0.052826,0.267805,0.0,0.0,0.678129,0.0,0.0,1.891366,0.22888,...,0.457143,0.042686,0.0,0.0,0.0,0.0,0.0,4.096988,0.073125,0.0,0.015152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4612,0.0,0.0,2.153026,0.0,0.0,66.4
2,38e29bc0-8fc0-11eb-b5f2-0742da91b282.jpg,0.0,0.0,0.001588,0.0,4.898275,0.0,0.0,1.250681,0.036835,0.0,0.0,0.0,1.215819,0.592546,5.262149,0.067028,0.265735,0.0,0.0,0.668743,0.0,0.0,1.917466,0.237922,...,0.412653,0.049392,0.0,0.0,0.0,0.0,0.0,4.134161,0.076137,0.0,0.017131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.444417,0.0,0.0,2.189022,0.0,0.0,63.2
3,6cd5d480-8b8b-11eb-b152-f7b115384fa9.jpg,0.0,0.0,0.000173,0.0,4.908416,0.0,0.0,1.26702,0.051824,0.0,0.0,0.0,1.207435,0.605955,5.256791,0.06513,0.266906,0.0,0.0,0.666541,0.0,0.0,1.939418,0.235997,...,0.407205,0.04816,0.0,0.0,0.0,0.0,0.0,4.132693,0.070295,0.0,0.022843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.420259,0.0,0.0,2.175637,0.0,0.0,87.1
4,76ed6fc0-8e1d-11eb-ad70-7be1a1219834.jpg,0.0,0.0,0.001794,0.0,4.928632,0.0,0.0,1.250752,0.036022,0.0,0.0,0.0,1.212313,0.586105,5.260494,0.060708,0.266331,0.0,0.0,0.676268,0.0,0.0,1.914697,0.241128,...,0.431022,0.046676,0.0,0.0,0.0,0.0,0.0,4.134078,0.080924,0.0,0.018746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.461155,0.0,0.0,2.197828,0.0,0.0,59.1
5,8d23fa60-9201-11eb-b3e6-cbfb48c6a300.jpg,0.0,0.0,0.0,0.0,4.913174,0.0,0.0,1.265471,0.058285,0.0,0.0,0.0,1.194981,0.592711,5.341297,0.096545,0.268521,0.0,0.0,0.671043,0.0,0.0,1.756436,0.278063,...,0.364584,0.016856,0.0,0.0,0.0,0.0,0.003713,4.169618,0.154638,0.0,0.028241,0.0,0.000196,0.0,0.0,0.0,0.000671,0.0,0.501747,0.0,0.0,2.107327,0.0,0.0,86.4
6,9dc3eba0-8d8d-11eb-8d3f-4776be867c61.jpg,0.0,0.0,0.0,0.0,4.798597,0.0,0.0,1.260125,0.049902,0.0,0.0,0.0,1.2358,0.588496,5.168211,0.090648,0.275804,0.0,0.004053,0.647746,0.0,0.0,1.716465,0.281806,...,0.423575,0.012758,0.000069,0.0,0.0,0.0,0.030556,4.030042,0.14944,0.0,0.026023,0.0,0.000627,0.0,0.0,0.0,0.005098,0.0,0.550922,0.0,0.0,2.063637,0.0,0.0,100.6
7,9dd020a0-8d8d-11eb-8d3f-bfd9f6bd04d4.jpg,0.0,0.0,0.0,0.0,4.870487,0.0,0.0,1.277971,0.044576,0.0,0.0,0.0,1.213791,0.658078,5.334865,0.096003,0.293912,0.0,0.000085,0.675494,0.0,0.0,1.734279,0.26623,...,0.395978,0.020612,0.0,0.0,0.0,0.0,0.029332,4.095998,0.108282,0.0,0.01738,0.0,0.000659,0.0,0.0,0.0,0.004383,0.0,0.498547,0.0,0.0,2.136361,0.0,0.0,106.1
8,d3757800-8dff-11eb-a328-472d4f8db277.jpg,0.0,0.0,0.0,0.0,5.007003,0.0,0.0,1.264733,0.048589,0.0,0.0,0.0,1.196861,0.6183,5.311005,0.140203,0.296374,0.0,0.002209,0.7,0.0,0.0,1.827778,0.29201,...,0.392334,0.009061,0.0,0.0,0.0,0.0,0.014609,4.082306,0.119844,0.0,0.024264,0.0,0.0,0.0,0.0,0.0,0.002515,0.0,0.588309,0.0,0.0,2.131566,0.0,0.0,99.9


In [17]:
# Selecting only the DL features cols
dl_cols = ["f"+ str(i) for i in range(1,2049)]
len(dl_cols)

2048

## **Generating_Predictions**

In [25]:
# Generating the predictions
y_preds = []
for i in range(0,9):
    selected_img = data_df[data_df['Img_name'] == data_df['Img_name'].iloc[i]][dl_cols]
    y_preds.append(loaded_rf.predict(selected_img.values)[0])

In [27]:
# Predictions 
y_preds

[86.4, 86.4, 86.4, 86.4, 86.4, 86.4, 86.4, 86.4, 86.4]

In [29]:
# Adding predictions column
data_df['Pred_Heights'] = y_preds

In [30]:
data_df

,Img_name,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,...,f2026,f2027,f2028,f2029,f2030,f2031,f2032,f2033,f2034,f2035,f2036,f2037,f2038,f2039,f2040,f2041,f2042,f2043,f2044,f2045,f2046,f2047,f2048,Height,Pred_Heights
0,1d5fee00-ada4-11eb-a80b-3f50af21830f.jpg,0.0,0.0,0.0,0.0,4.919989,0.0,0.0,1.232318,0.041064,0.0,0.0,0.0,1.211448,0.596881,5.314107,0.068448,0.270772,0.0,0.0,0.671996,0.0,0.0,1.887564,0.23478,...,0.046488,0.0,0.0,0.0,0.0,0.0,4.125062,0.069655,0.0,0.016122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.443612,0.0,0.0,2.12557,0.0,0.0,76.7,86.4
1,1eddf4c0-ad9f-11eb-a80b-ef4adba049aa.jpg,0.0,0.0,0.0,0.0,4.897092,0.0,0.0,1.242595,0.040788,0.0,0.0,0.0,1.19496,0.58566,5.247516,0.052826,0.267805,0.0,0.0,0.678129,0.0,0.0,1.891366,0.22888,...,0.042686,0.0,0.0,0.0,0.0,0.0,4.096988,0.073125,0.0,0.015152,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.4612,0.0,0.0,2.153026,0.0,0.0,66.4,86.4
2,38e29bc0-8fc0-11eb-b5f2-0742da91b282.jpg,0.0,0.0,0.001588,0.0,4.898275,0.0,0.0,1.250681,0.036835,0.0,0.0,0.0,1.215819,0.592546,5.262149,0.067028,0.265735,0.0,0.0,0.668743,0.0,0.0,1.917466,0.237922,...,0.049392,0.0,0.0,0.0,0.0,0.0,4.134161,0.076137,0.0,0.017131,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.444417,0.0,0.0,2.189022,0.0,0.0,63.2,86.4
3,6cd5d480-8b8b-11eb-b152-f7b115384fa9.jpg,0.0,0.0,0.000173,0.0,4.908416,0.0,0.0,1.26702,0.051824,0.0,0.0,0.0,1.207435,0.605955,5.256791,0.06513,0.266906,0.0,0.0,0.666541,0.0,0.0,1.939418,0.235997,...,0.04816,0.0,0.0,0.0,0.0,0.0,4.132693,0.070295,0.0,0.022843,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.420259,0.0,0.0,2.175637,0.0,0.0,87.1,86.4
4,76ed6fc0-8e1d-11eb-ad70-7be1a1219834.jpg,0.0,0.0,0.001794,0.0,4.928632,0.0,0.0,1.250752,0.036022,0.0,0.0,0.0,1.212313,0.586105,5.260494,0.060708,0.266331,0.0,0.0,0.676268,0.0,0.0,1.914697,0.241128,...,0.046676,0.0,0.0,0.0,0.0,0.0,4.134078,0.080924,0.0,0.018746,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.461155,0.0,0.0,2.197828,0.0,0.0,59.1,86.4
5,8d23fa60-9201-11eb-b3e6-cbfb48c6a300.jpg,0.0,0.0,0.0,0.0,4.913174,0.0,0.0,1.265471,0.058285,0.0,0.0,0.0,1.194981,0.592711,5.341297,0.096545,0.268521,0.0,0.0,0.671043,0.0,0.0,1.756436,0.278063,...,0.016856,0.0,0.0,0.0,0.0,0.003713,4.169618,0.154638,0.0,0.028241,0.0,0.000196,0.0,0.0,0.0,0.000671,0.0,0.501747,0.0,0.0,2.107327,0.0,0.0,86.4,86.4
6,9dc3eba0-8d8d-11eb-8d3f-4776be867c61.jpg,0.0,0.0,0.0,0.0,4.798597,0.0,0.0,1.260125,0.049902,0.0,0.0,0.0,1.2358,0.588496,5.168211,0.090648,0.275804,0.0,0.004053,0.647746,0.0,0.0,1.716465,0.281806,...,0.012758,0.000069,0.0,0.0,0.0,0.030556,4.030042,0.14944,0.0,0.026023,0.0,0.000627,0.0,0.0,0.0,0.005098,0.0,0.550922,0.0,0.0,2.063637,0.0,0.0,100.6,86.4
7,9dd020a0-8d8d-11eb-8d3f-bfd9f6bd04d4.jpg,0.0,0.0,0.0,0.0,4.870487,0.0,0.0,1.277971,0.044576,0.0,0.0,0.0,1.213791,0.658078,5.334865,0.096003,0.293912,0.0,0.000085,0.675494,0.0,0.0,1.734279,0.26623,...,0.020612,0.0,0.0,0.0,0.0,0.029332,4.095998,0.108282,0.0,0.01738,0.0,0.000659,0.0,0.0,0.0,0.004383,0.0,0.498547,0.0,0.0,2.136361,0.0,0.0,106.1,86.4
8,d3757800-8dff-11eb-a328-472d4f8db277.jpg,0.0,0.0,0.0,0.0,5.007003,0.0,0.0,1.264733,0.048589,0.0,0.0,0.0,1.196861,0.6183,5.311005,0.140203,0.296374,0.0,0.002209,0.7,0.0,0.0,1.827778,0.29201,...,0.009061,0.0,0.0,0.0,0.0,0.014609,4.082306,0.119844,0.0,0.024264,0.0,0.0,0.0,0.0,0.0,0.002515,0.0,0.588309,0.0,0.0,2.131566,0.0,0.0,99.9,86.4


In [32]:
# Storing the final results
data_df['Pred_Heights'].to_csv("Pred_Heights.csv")

In [34]:
# copy it there
!cp Pred_Heights.csv /content/drive/MyDrive/CGM_Assignment

## **SUMMARY**

- Here, we have generated the DL CNN based features for all the given images.
    - Then, used these features to train a RF Regressor model for yielding the children heights.